---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [61]:
import pandas as pd

from time import strptime
from datetime import datetime

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
pd.set_option("display.max_rows", None, "display.max_columns", None)
df = pd.Series(doc)
df.head(462)

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [67]:
import re

def date_sorter():
    global df
    frame = pd.Series(df)
    months = {'january':1, 'february':2, 'march':3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 'september':9, 'october':10, 'november':11, 'december':12}
    listo= []
    
    index = 0
    for i in df:
#        print(index)
        dicto = {'day':1,'month':1, 'year':1900}
        result = re.search(r'''(?P<Date>(?P<Month>([1-9])|(1[0-2]))[-/](?P<Day>\d?\d)[/-](?P<Year>\d?\d?\d\d))''', i)
        result_2 = re.search(r'''(?P<Date>(?P<Month>[a-zA-Z]{3,})\.?-? ?(?P<Day>\d\d?)(st|nd|th)?,?-? ?(?P<Year>\d{4}))''', i)
        result_3 = re.search(r'''(?P<Date>(?P<Day>([0-9])|(1[0-9])|(2[0-9])|(3[0-1]))-?/?,?\s?(?P<Month>[A-Za-z]{3,10})-?/?,?\s?(?P<Year>\d{2,4}))''', i)
        result_4 = re.search(r'''(?P<Date>(?P<Month>[A-Z][a-z]{2,}),?\.? (?P<Year>\d{4}))''', i)
        result_5 = re.search(r'''(?P<Date>(?P<Month>\d\d?)/(?P<Year>\d{4}))''', i)
        result_6 = re.search(r'''(?P<Date>(?P<year>\d{4}))''', i)
        if result != None:
#            print(result.group('Date'))
#            print('Result11111111111111111111111111111111111111111111111111111111111')
            dicto['day'] = int(result.group('Day'))
            dicto['month'] = int(result.group('Month'))
            if len(result.group('Year'))<3:
                dicto['year'] = int(result.group('Year'))+1900
            else:
                dicto['year'] = int(result.group('Year'))
            listo.append(dicto)
        
        elif result_2 != None and result_2.group('Month').lower()[:3] in [month.lower()[:3] for month in months.keys()]:#
#            print('Result222222222222222222222222222222222222222222222222222222222222222222222')
            #print(type(result_2))
 #           print(result_2.group('Date'))
            #print(result_2)
        
            for month in months.keys():
                if result_2.group(0)[:3].lower() in month:

                    dicto['day'] = int(result_2.group('Day'))
                    dicto['month'] = months[month]
                    dicto['year'] = int(result_2.group('Year'))
                    listo.append(dicto)


        elif result_3 != None and result_3.group('Month').lower()[:3] in [month.lower()[:3] for month in months.keys()]:
  #          print('Result3333333333333333333333333333333333333333333333333333333333333333333')
   #         print(result_3.group('Date'))
            
            dicto['day'] = int(result_3.group('Day'))
            for month in months.keys():
                if result_3.group('Month')[:3].lower() in month:
                    dicto['month'] = months[month]
            dicto['year'] = int(result_3.group('Year'))        
            listo.append(dicto)
                                 

        elif result_4 != None and result_4.group('Month').lower()[:3] in [month.lower()[:3] for month in months.keys()]:
    #        print('Result444444444444444444444444444444444444444444444444444444444444444444')
            #print(type(result_4))
            #print(result_4)
     #       print(result_4.group('Date'))
            
            for month in months.keys():
                
                if result_4.group('Month').lower()[:3] in month.lower():
                    dicto['month'] = months[month]
                elif type(result_4.group('Month'))== int:
                    dicto['month'] = int(result_4.group('Month'))

            if len(result_4.group('Year'))<3:
                dicto['year'] = int(result_4.group('Year'))+1900
            else:
                dicto['year'] = int(result_4.group('Year'))
            listo.append(dicto)
                                 
                
        elif result_5 != None:
      #      print('Result55555555555555555555555555555555555555555555555555555555555555555555')
       #     print(result_5.group('Date'))
            dicto['month'] = int(result_5.group('Month'))
            if len(result_5.group('Year'))<3:
                dicto['year'] = int(result_5.group('Year'))+1900
            else:
                dicto['year'] = int(result_5.group('Year'))
            listo.append(dicto)
            
        elif result_6 != None:
        #    print('Result66666666666666666666666666666666666666666666666666666666666666666666')#
#            print(result_6.group('Date'))
            dicto['year'] = int(result_6.group('year'))
            listo.append(dicto)                                
        #else:
            #print('________________________________YOU FAILED________________________________________')
        #print(index)
        #print(listo[index])
        index +=1


    data = pd.DataFrame.from_dict(listo)
    #datasort = data.sort_values(by='year')
    datadate = pd.to_datetime(data, infer_datetime_format=True)
    datasort = datadate.sort_values()
    #print(len(data), len(datadate), len(datasort))
    #print(len(listo))
    #print(listo)
    
    final = pd.Series(datasort.index.tolist())
    return final
date_sorter()

500


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     488
70     303
71     283
72     395
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     369
82     493
83     318
84     239
85     148
86     105
87     336
88       6
89     200
90      81